# خلف الكواليس (باستخدام بايتورتش)

قم بتثبيت مكتبات

transformers, datasets, evaluate

لتشغيل هذا الدفتر.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [43]:
from transformers import pipeline

sentimentAnalysisModelName = 'MaagDeveloper/CAMeL-Lab-arabic-sentiment-analysis'
classifier = pipeline("sentiment-analysis", model=sentimentAnalysisModelName)
classifier(
    [
        "لقد كنتُ أترقّب الدورة من هاجِّنغ فيس طوال حياتي.",
        "أكره هذا بشدّة!"
    ]
)

Device set to use cpu


[{'label': 'positive', 'score': 0.9497042894363403},
 {'label': 'negative', 'score': 0.9927158951759338}]

تحميل محول النصوص

ال tokenizer

هو المسؤول عن تحويل الكلام لأرقام لان المحولات لا تفهم الكلام ولكن الأرقام فقط.

In [45]:
from transformers import AutoTokenizer
checkpoint = "MaagDeveloper/CAMeL-Lab-arabic-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

لو لاحظنا، الجملتين مش بنفس الطول – واحدة أطول من الثانية. وهنا عندنا خيارين عشان نخليهم بنفس الطول:

1. الحشو (Padding): نضيف رموز إضافية (غالباً أصفار) في نهاية الجمل الأقصر عشان توصل لطول الجملة الأطول.

2. الاقتطاع (Truncation): نقطع الجمل الأطول ونخليها بنفس طول الجمل الأقصر أو الطول اللي نحدده.

الهدف إن النموذج ياخذ مدخلات بطول موحد، لأن النماذج ما تشتغل بكفاءة على جمل بطول متغير.

# مثال 1: الحشو فقط

In [46]:
raw_inputs = [
    "لقد كنتُ أترقّب الدورة من هاجِّنغ فيس طوال حياتي.",
    "أكره هذا بشدّة!"
]

inputs = tokenizer(raw_inputs, padding=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[    2,  3863,     1,     1,  6371,  1908,     1, 11256,  8026,  5097,
            18,     3],
        [    2,  5983,  2197,  2085,     1,     5,     3,     0,     0,     0,
             0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]])}


# مثال 2: الحشو + الاقتطاع

In [47]:

raw_inputs = [
    "لقد كنتُ أترقّب الدورة من هاجِّنغ فيس طوال حياتي.",
    "أكره هذا بشدّة!"
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, max_length=5, return_tensors="pt")
print(inputs)


{'input_ids': tensor([[   2, 3863,    1,    1,    3],
        [   2, 5983, 2197, 2085,    3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]])}


# مثال 3: الحشو لعدد معين دائمًا

In [48]:

raw_inputs = [
    "لقد كنتُ أترقّب الدورة من هاجِّنغ فيس طوال حياتي.",
    "أكره هذا بشدّة!"
]
inputs = tokenizer(raw_inputs, padding='max_length', truncation=True, max_length=5, return_tensors="pt")
print(inputs)


{'input_ids': tensor([[   2, 3863,    1,    1,    3],
        [   2, 5983, 2197, 2085,    3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]])}


# تحميل المحول نفسه

In [50]:
from transformers import AutoModel
checkpoint = "MaagDeveloper/CAMeL-Lab-arabic-sentiment-analysis"
model = AutoModel.from_pretrained(checkpoint)

 شكل (أبعاد) تمثيل الكلمات داخل النموذج.

last_hidden_state هو مصفوفة فيها التمثيلات النهائية لكل كلمة في الجملة.

الشكل بيكون: (batch_size, sequence_length, hidden_size)

يعني: عدد الجمل × طول الجملة (عدد التوكنز) × عدد الخصائص (الأبعاد) لكل كلمة.

In [51]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 5, 768])


# أول حاجة بنجيب موديل جاهز للتصنيف من مكتبة

transformers:

In [52]:
from transformers import AutoModelForSequenceClassification

بعدين بنحدد نقطة التحميل (checkpoint) للموديل اللي مدربينه على تحليل المشاعر بالعربي:

In [53]:
checkpoint = "MaagDeveloper/CAMeL-Lab-arabic-sentiment-analysis"


نحمّل الموديل الجاهز ده:

In [54]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

بعدين ندخل عليه البيانات اللي جهزناها (inputs) عشان يطلع النتائج:

In [55]:
outputs = model(**inputs)

هنا بنطبع شكل النواتج:

logits دي الأرقام الخام اللي النموذج حسبها قبل ما نطلع منها التوقع النهائي.

شكلها بيكون (batch_size, num_labels) يعني: عدد الجمل × عدد الفئات اللي بنصنفها (مثلاً إيجابي، سلبي، محايد).

In [56]:
print(outputs.logits.shape)

torch.Size([2, 3])


القيم نفسها:

دي الأرقام اللي النموذج حسبها لكل فئة قبل ما نطبق عليها دالة مثل softmax لتحويلها لاحتمالات.

In [57]:
print(outputs.logits)

tensor([[ 0.0743, -0.7427, -0.4310],
        [-2.6578,  3.5516, -1.4452]], grad_fn=<AddmmBackward0>)


أول شيء بنستخدم دالة

softmax

لتحويل الأرقام الخام

(logits)

اللي طلعها النموذج إلى احتمالات فعلية:


ليه softmax

لأن

logits

هي أرقام ممكن تكون موجبة أو سالبة، مش مقيّدة بين 0 و1، ومش بتعبر عن احتمال.

دالة

softmax

بتحولهم لمجموعة احتمالات مجموعها 1 لكل جملة، يعني كل رقم صار يمثل احتمال إن الجملة تنتمي لكل فئة.

dim=-1

 معناها بنطبق العملية على آخر بعد في المصفوفة، اللي هو عدد الفئات.



In [58]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)


tensor([[0.4890, 0.2160, 0.2950],
        [0.0020, 0.9913, 0.0067]], grad_fn=<SoftmaxBackward0>)


دي حاجة موجودة في إعدادات الموديل، وتربط بين أرقام الفئات وتسمياتها الحقيقية.

يعني بدل ما تشوف رقم زي 0 أو 1 أو 2، تقدر تعرف إذا الرقم ده يعني "سلبي" أو "محايد" أو "إيجابي".

ببساطة، هي "قاموس" بيربط كل رقم تصنيف بالاسم أو الوصف بتاعه.

تقدر تستخدمها عشان تفهم نتائج النموذج بشكل أوضح.

In [59]:
model.config.id2label

{0: 'positive', 1: 'negative', 2: 'neutral'}